In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [198]:
direc = '../../../../Dropbox/Summer Project 2020 data/epi_ridge_cut/'
output = '../dataFolder/AmplitudeAnalysis/epi_ridge_cut/'

In [188]:
# read a random fly and check the length errors
# mode = 
files = glob.glob(direc + 'fly*')

In [53]:
# read the text file as a dictionary

Nomenclature = pd.read_csv(direc + 'data_organization_epi.txt')

d = {}
with open(direc + 'data_organization_epi.txt') as f:
    for line in f:
        (key, val) = line.split('=')
        d[key] = val.strip()

mode = d['mode']
del d['mode']

for k in d:
    d[k] = int(d[k])

In [54]:
d

{'lwapex': 1,
 'rwapex': 2,
 'lwbase': 3,
 'rwbase': 4,
 'fixed': 5,
 'lhapex': 6,
 'rhapex': 7,
 'lhbase': 8,
 'rhbase': 9}

In [63]:
lengthCheckFor = ['lw', 'rw', 'lh', 'rh']

In [64]:
def SliceColumnsForLocation(dictionary, df, bodypart, location):
    k = [key for key in dictionary if bodypart + location in key][0]
    index = dictionary[k]

    pt1 = df.iloc[:, 3*index - 2 - 1 : 3*index]
    pt1.columns = ['x', 'y', 'z']
    return(pt1)

In [203]:
for ff in files:
    flyname = os.path.basename(ff)
    
    treatments  = glob.glob(ff + '/*/*xyzpts.csv')
    
    for readThis in treatments: 
    
        df = pd.read_csv(readThis)

        a = os.path.basename(readThis).split('_')[2]
        b = os.path.basename(readThis).split('_')[3]
        winglength = a + '_' + b

        # extract teh vectors for all the body parts 

        # make a pandas to hold all the vectors
        midx = pd.MultiIndex.from_product([lengthCheckFor, ['x', 'y', 'z', 'length']])
        uber_df = pd.DataFrame(data = None, columns = midx)
        uber_df.head()

        #loop through all body parts
        for part in lengthCheckFor:
            base = SliceColumnsForLocation(d, df, part, 'base')
            tip = SliceColumnsForLocation(d, df, part, 'apex')
            pt = tip - base

            length =np.linalg.norm(pt, axis = 1, ord = 2)
            pt['length'] = length
            uber_df.loc[:,(part, 'x')] = pt['x']
            uber_df.loc[:,(part, 'y')] = pt['y']
            uber_df.loc[:,(part, 'z')] = pt['z']
            uber_df.loc[:,(part, 'length')] = pt['length']

        # save to folder
        filename = flyname + '_' + winglength 
        uber_df.to_csv(output +  filename + '_WingHaltereVectors.csv' )

        # compute errors in lengths    
        Results = pd.DataFrame(data = None
                       , index = ['mean', 'std', 'mean Abs dev', 'mean % dev']
                       , columns = lengthCheckFor)

        for part in lengthCheckFor:
            m = uber_df.loc[:,(part,'length')].mean(skipna = True)
            s = uber_df.loc[:,(part,'length')].std(skipna = True)
            mad = uber_df.loc[:,(part,'length')].mad(skipna = True)
            pc = mad/m * 100

            Results.loc['mean', part] = m
            Results.loc['std', part] = s
            Results.loc['mean Abs dev', part] = mad
            Results.loc['mean % dev', part] = pc
        Results.to_csv(output + filename + '_LengthError.csv')